In [7]:
import pandas as pd
import numpy as np

from pandas_profiling import ProfileReport
import plotly.express as px
import re

from itertools import groupby
from datetime import datetime
import math

#import re
#from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer

#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

import networkx as nx

from networkx.readwrite import json_graph

import seaborn as sns

import matplotlib.pyplot as plt

import zipfile

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 50

In [253]:
df_spec = pd.read_excel("../data/external/Exhibit C_Leveled Bid - Specs_RFP.xlsx", 
                   sheet_name='Serenity Leveled Bid Request',
                   header=12)

df_spec.drop(['Unnamed: 0'], axis=1, inplace=True)
df_spec.fillna('DNA', inplace=True) ## DNA: Does Not Apply
df_spec.shape

(85, 26)

In [254]:
upd_ix = [(x,y) for (x,y) in enumerate(df_spec.Area) if y != 'DNA']
##upd_ix

In [255]:
for i in range(len(upd_ix) - 1):
    df_spec.Area.iloc[upd_ix[i][0]: upd_ix[i+1][0]] = upd_ix[i][1]

i = i+1
df_spec.Area.iloc[upd_ix[i][0]:] = upd_ix[i][1]

In [256]:
df_spec = df_spec[['Area', 'Cost center', 'Item', 'Category', 'SKU/Spec', 'Color/Notes',
       'Reference']]

ix = df_spec.iloc[:,1:].apply(lambda x: np.sum(x == 'DNA') != len(x), axis=1)
df_spec = df_spec.loc[ix==True,:]

df_spec.head(2)

,Area,Cost center,Item,Category,SKU/Spec,Color/Notes,Reference
0,Kitchen,3-05-5410,Cabinets,Material,5pc all wood shaker style cabinet - door & dra...,White above / Gray below,DNA
1,Kitchen,3-05-5410,Cabinets Install,Labor,"Remove existing doors, prep+paint boxes, insta...",DNA,DNA


In [212]:
x = df_spec.Area.value_counts()
##for index, value in zip(x.index, x):
##    print (index, value)
x

Kitchen                            22
Bath                               15
Overhead                           10
Floors                              9
Paint & Other Misc Living Areas     8
Repairs                             4
General Conditions                  4
Windows                             1
Name: Area, dtype: int64

In [213]:
arr = []

for index, value in zip(x.index, x):
    arr.extend([[index, y] for y in np.linspace(1,value, value).astype(int)])
    
idx_df = pd.DataFrame(arr, columns=['Area', 'idx'])
idx_df.sort_values(by=['Area', 'idx'], inplace=True)
idx_df.reset_index(drop=True, inplace=True)
idx_df.shape

(73, 2)

In [258]:
df_spec.sort_values(by=['Area', 'Cost center', 'Item'], inplace=True)
df_spec.reset_index(drop=True, inplace=True)

df_spec['idx'] = idx_df.idx
df_spec.head(2)

,Area,Cost center,Item,Category,SKU/Spec,Color/Notes,Reference,idx
0,Bath,3-05-5260,"Interior Finish Work, Other",Labor,Bath Finish Accessories & Misc Carpentry - labor,DNA,DNA,1
1,Bath,3-05-5270,Bath / Shower,Material,Water-Saving Massage Showerhead,"Brushed Nickel, Brass Ball Joint",LINK,2


In [440]:
def prep_bid(df_in):
    df = df_in.copy()
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    df.fillna('DNA', inplace=True) ## DNA: Does Not Apply
    
    upd_ix = [(x,y) for (x,y) in enumerate(df.Area) if y != 'DNA']
    
    for i in range(len(upd_ix) - 1):
        df.Area.iloc[upd_ix[i][0]: upd_ix[i+1][0]] = upd_ix[i][1]

    i = i+1
    df.Area.iloc[upd_ix[i][0]:] = upd_ix[i][1]
    
    df = df.loc[df['Cost center'] != 'DNA',:]
    
    df.sort_values(by=['Area', 'Cost center', 'Item'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    x = df_spec.Area.value_counts()
    
    arr = []

    for index, value in zip(x.index, x):
        arr.extend([[index, y] for y in np.linspace(0,value-1, value-1).astype(int)])

    idx_df = pd.DataFrame(arr, columns=['Area', 'idx'])
    idx_df.sort_values(by=['Area', 'idx'], inplace=True)
    idx_df.reset_index(drop=True, inplace=True)
    
    df['idx'] = idx_df.idx
    
    cols_q = [x for x in np.where([x.startswith('Quantity') for x in df.columns])[0]]

    cols = cols_q.copy()

    cols.extend([x + 1 for x in cols_q])
    cols.extend([x + 2 for x in cols_q])
    cols.sort()
    
    k_cols = [0, 1, 2, 3, np.where(['idx' in x for x in df.columns])[0][0]]
    
    lst = list()
    repeat = sum(['Quantity' in x for x in df.columns])
    for i in range(repeat):
        c_cols = k_cols.copy()
        c_cols.extend(cols[3*i:3*i+3])
        ##print(c_cols)
        ##print(i)
        temp_df = pd.DataFrame(df.iloc[:,c_cols]).copy()
        temp_df.columns = ['area', 'cost_center', 'item', 'category', 'idx', 'qty', 'unit','price']
        temp_df.loc[temp_df.price == 'DNA', 'price'] = 0
        temp_df.loc[:, 'price'] = [np.round(x,2) for x in temp_df.price]
        
        temp_df.loc[temp_df.qty.isin(['DNA', 'TBD']), 'qty'] = 0
        temp_df.loc[:, 'qty'] = [np.round(x) for x in temp_df.qty]
        
        temp_df['sp_type'] = 'spec_type_' + str(i)
        
        lst.append(temp_df)
        
    df = pd.concat(lst, axis=0)
    
    ##df.columns = ['area', 'cost_center', 'item', 'category', 'idx', 'qty', 'unit','price', 'spec_type']
    return(df)
    

In [441]:
df_bid_rams = pd.read_excel("../data/external/RFP Cost Comparison_Tracker_v9_3.0 Submission 1.6.21.xlsx", 
                   sheet_name='R.A.M.S 2.0', header=6)

df_bid_rams = prep_bid(df_bid_rams)
df_bid_rams['vendor'] = 'RAMS'
df_bid_rams.reset_index(drop=True, inplace=True)

In [426]:
##df_bid_katerra = pd.read_excel("../data/external/RFP Cost Comparison_Tracker_v9_3.0 Submission 1.6.21.xlsx", 
##                   sheet_name='Katerra', header=12)

#df_bid_katerra = prep_bid(df_bid_katerra)
#df_bid_katerra['vendor'] = 'Katerra'
##df_bid_katerra.head()

In [443]:
df_bid_pro_craft = pd.read_excel("../data/external/RFP Cost Comparison_Tracker_v9_3.0 Submission 1.6.21.xlsx", 
                   sheet_name='Pro-Craft 2.0', header=12)

df_bid_pro_craft = prep_bid(df_bid_pro_craft)
df_bid_pro_craft['vendor'] = 'Pro-Craft 2.0'
df_bid_pro_craft.reset_index(drop=True, inplace=True)

df_bid_pro_craft.head()

,area,cost_center,item,category,idx,qty,unit,price,sp_type,vendor
0,Bath,3-05-5260,"Interior Finish Work, Other",Labor,0.0,2.0,Count,0.00,spec_type_0,Pro-Craft 2.0
1,Bath,3-05-5270,Bath / Shower,Material,1.0,2.0,Count,0.00,spec_type_0,Pro-Craft 2.0
2,Bath,3-05-5430,Vanity,Material,2.0,11.0,Door/Drawer,0.00,spec_type_0,Pro-Craft 2.0
3,Bath,3-05-5430,Vanity,Labor,3.0,11.0,Door/Drawer,955.43,spec_type_0,Pro-Craft 2.0
4,Bath,3-05-5450,Vanity top / sink,Material,4.0,72.0,LNIN,0.00,spec_type_0,Pro-Craft 2.0


In [444]:
final_df = pd.concat([df_bid_rams, df_bid_pro_craft], axis=0)
final_df.reset_index(drop=True, inplace=True)
final_df.shape

(720, 10)

In [1]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://localhost/TB')

In [446]:
final_df.to_sql('lvl_bid', con=engine, index=False, if_exists='replace')

In [447]:
##[x for s in final_df.price if x is not float]
final_df.shape

(720, 10)

In [448]:
final_df.to_csv("../data/processed/lvl_bid_postgres_load.csv", header=True, index=False)

In [2]:
import psycopg2

In [3]:
conn = psycopg2.connect(host="localhost", port = 5432, database="TB")

In [4]:
cur = conn.cursor()

In [12]:
cur.execute("""SELECT * FROM lvl_bid""")
query_results = cur.fetchall()
query_results

[('Bath',
  '3-05-5260',
  'Interior Finish Work, Other',
  'Labor',
  0.0,
  2.0,
  'Count',
  300.33,
  'spec_type_0',
  'RAMS'),
 ('Bath',
  '3-05-5270',
  'Bath / Shower',
  'Material',
  1.0,
  2.0,
  'Count',
  0.0,
  'spec_type_0',
  'RAMS'),
 ('Bath',
  '3-05-5430',
  'Vanity',
  'Material',
  2.0,
  11.0,
  'Door/Drawer',
  592.74,
  'spec_type_0',
  'RAMS'),
 ('Bath',
  '3-05-5430',
  'Vanity',
  'Labor',
  3.0,
  11.0,
  'Door/Drawer',
  523.26,
  'spec_type_0',
  'RAMS'),
 ('Bath',
  '3-05-5450',
  'Vanity top / sink',
  'Material',
  4.0,
  72.0,
  'LNIN',
  211.38,
  'spec_type_0',
  'RAMS'),
 ('Bath',
  '3-05-5450',
  'Vanity top / sink',
  'Labor',
  5.0,
  72.0,
  'LNIN',
  563.92,
  'spec_type_0',
  'RAMS'),
 ('Bath',
  '3-05-5450',
  'Vanity top / sink',
  'Material',
  6.0,
  2.0,
  'Count',
  149.1,
  'spec_type_0',
  'RAMS'),
 ('Bath',
  '3-05-5470',
  'Mirror',
  'Material',
  7.0,
  2.0,
  'Count',
  0.0,
  'spec_type_0',
  'RAMS'),
 ('Bath',
  '3-05-5620',
  'F

In [34]:
df_summ = pd.read_sql_query("SELECT vendor, area, sum(price) FROM lvl_bid group by vendor, area", conn)
df_unit = pd.read_sql_query("SELECT vendor, sp_type, area, sum(price) FROM lvl_bid group by vendor, sp_type, area", conn)

In [35]:
##df.to_json()

In [36]:
units = df_unit.sp_type.unique()
units = np.hstack([units, ['All Units']])
units.sort()
units[0]

'All Units'

In [33]:
mask = df["sp_type"] == 'spec_type_3'
##mask

In [37]:
import plotly.express as px
fig = px.bar(df_summ, x="vendor", y="sum", color="area", barmode="group")
fig.show()

In [40]:
mask = df_unit["sp_type"] == units[1]
fig = px.bar(df_unit[mask], x="vendor", y="sum", color="area", barmode="group")
fig.show()